In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
2,application_1607531848319_0003,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
from pyspark.ml.feature import VectorAssembler,MinMaxScaler,StandardScaler
from pyspark.ml import Pipeline
from pyspark.ml.classification import RandomForestClassifier,RandomForestClassificationModel
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Read the data && data preprocessing

In [3]:
def data_load_and_transform(path):
  dataset = spark.read.option( 'delimiter',';').csv(path,inferSchema=True, header =True)
  new_columns_name_list=list(map(lambda x : x.replace('"',''),dataset.columns))
  dataset=dataset.toDF(*new_columns_name_list)
  feature_select_list=[dataset.columns[i] for i in [10,9,1,2]]##### Feature selection
  feature_list = []
  for col in dataset.columns:
    # if col == 'quality':
    if col not in feature_select_list:
      continue
    else:
      feature_list.append(col)

  assembler=VectorAssembler(inputCols=feature_list,outputCol='features')
  df=assembler.transform(dataset)
  return df

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
dataset='s3://dataset-cs643-proj/TrainingDataset.csv'
dataset=data_load_and_transform(dataset)
test='s3://dataset-cs643-proj/ValidationDataset.csv'
test=data_load_and_transform(test)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Setup multiple threads for later training

In [13]:
from multiprocessing.pool import ThreadPool
pool=ThreadPool(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Model train && predict

In [19]:
import timeit
start_time = timeit.default_timer()

rf = RandomForestClassifier(labelCol='quality', featuresCol='features',\
                            numTrees=20,maxDepth=6,seed=10,impurity='entropy')

model=rf.fit(dataset)
predictions=model.transform(test)

evaluator= MulticlassClassificationEvaluator(labelCol='quality',predictionCol='prediction',\
                                             metricName='f1')
accuracy=evaluator.evaluate(predictions)

print(f'F1 score: {accuracy}')
print('Test Error = %g' % (1.0-accuracy))
elapsed = timeit.default_timer() - start_time
print(f'Runing Time: {round(elapsed)}s')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

F1 score: 0.603021582733813
Test Error = 0.396978
Runing Time: 13s

### Output the trained model for future testing

In [23]:
model.save('s3://dataset-cs643-proj/trainingModel')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Docker container part II

In [24]:
from pyspark.ml.classification import RandomForestClassifier,RandomForestClassificationModel
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import VectorAssembler

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Load the pretrained model 

In [32]:
model_load=RandomForestClassificationModel.load('s3://dataset-cs643-proj/trainingModel')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [25]:
def data_load_and_transform(path):
  dataset = spark.read.option( 'delimiter',';').csv(path,inferSchema=True, header =True)
  new_columns_name_list=list(map(lambda x : x.replace('"',''),dataset.columns))
  dataset=dataset.toDF(*new_columns_name_list)
  feature_select_list=[dataset.columns[i] for i in [10,9,1,2]]##### Feature selection
  feature_list = []
  for col in dataset.columns:
    # if col == 'quality':
    if col not in feature_select_list:
      continue
    else:
      feature_list.append(col)

  assembler=VectorAssembler(inputCols=feature_list,outputCol='features')
  df=assembler.transform(dataset)
  return df

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### The path can change to testingDataset.csv, just for showing the validation result

In [28]:
path='s3://dataset-cs643-proj/ValidationDataset.csv'

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [33]:
test=data_load_and_transform(path)
predictions=model_load.transform(test)
evaluator= MulticlassClassificationEvaluator(labelCol='quality',predictionCol='prediction',metricName='f1')
accuracy=evaluator.evaluate(predictions)

print(f'F1 score: {accuracy}')
print('Test Error = %g' % (1.0-accuracy))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

F1 score: 0.603021582733813
Test Error = 0.396978

In [34]:
print(f'Final predictions:')

print(predictions.select('prediction').show(10000))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Final predictions:
+----------+
|prediction|
+----------+
|       5.0|
|       5.0|
|       5.0|
|       6.0|
|       5.0|
|       5.0|
|       5.0|
|       5.0|
|       5.0|
|       6.0|
|       5.0|
|       6.0|
|       5.0|
|       5.0|
|       5.0|
|       5.0|
|       6.0|
|       5.0|
|       5.0|
|       6.0|
|       6.0|
|       5.0|
|       5.0|
|       5.0|
|       5.0|
|       6.0|
|       5.0|
|       5.0|
|       5.0|
|       6.0|
|       6.0|
|       5.0|
|       5.0|
|       5.0|
|       5.0|
|       6.0|
|       5.0|
|       5.0|
|       5.0|
|       5.0|
|       5.0|
|       5.0|
|       6.0|
|       6.0|
|       5.0|
|       6.0|
|       5.0|
|       6.0|
|       5.0|
|       5.0|
|       6.0|
|       7.0|
|       5.0|
|       6.0|
|       7.0|
|       5.0|
|       6.0|
|       6.0|
|       6.0|
|       7.0|
|       6.0|
|       6.0|
|       6.0|
|       6.0|
|       6.0|
|       6.0|
|       5.0|
|       5.0|
|       5.0|
|       5.0|
|       5.0|
|       6.0|
|     